In [1]:
# download dataset and unzip
# ! wget https://dev.fungismash.secondarymetabolites.org/upload/fungi-28f77eb4-83c8-400e-8b6c-9f06a0c99790/GCA_003184865.1_Asphom1_genomic.zip -P data/
# ! unzip data/GCA_003184865.1_Asphom1_genomic.zip -d data/antismash-result

In [2]:
# load library & data
import pandas as pd
import re
from os import listdir
from Bio import SeqIO
import numpy as np
# GET SUMMARY
df = pd.read_html('data/antismash-result/index.html')

In [3]:
# explore resulting list
# df[0-40] individual region
# df[41] summary
#print(len(df_summary))
# df[41]

def mibig(n, mode='region'):
    # MIBIG
    # Start from 42 + index (0-78) x 2
    #n = 78
    # MIBIG PROTOCLUSTER TO REGION
    if mode == 'region':
        x = 42 + (n*2)
    else:
        # MIBIG REGION TO REGION
        x = 42 + (n*2) + 1
    return x

In [4]:
# https://dev.fungismash.secondarymetabolites.org/upload/fungi-28f77eb4-83c8-400e-8b6c-9f06a0c99790/index.html
def print_cluster(index, df, type_gb, term, search = False):
    region = df.Region[index][-5:].split('.')
    region[0] = re.sub("[^0-9]", "", region[0])
    location = df.loc[index, 'gbk']
    path = "data/antismash-result/" + location
    for seq_record in SeqIO.parse(path, "genbank"):
        for seq_feat in seq_record.features:
            if seq_feat.type == type_gb:
                try:
                    for i in seq_feat.qualifiers[term]:
                        
                        if search != False:
                            if search in i:
                                print('index :'+str(index))
                                print(region)
                                print('https://dev.fungismash.secondarymetabolites.org/upload/fungi-28f77eb4-83c8-400e-8b6c-9f06a0c99790/index.html#r'+region[0]+'c'+region[1])    
                                print(i)
                        else:
                            #print('https://dev.fungismash.secondarymetabolites.org/upload/fungi-28f77eb4-83c8-400e-8b6c-9f06a0c99790/index.html#r'+region[0]+'c'+region[1])    
                            print(index, region[0]+"-"+region[1], i) 
                except:
                    pass
    
    return

In [5]:
df_summary = df[41] # list of all simplified

# add path to gbk region
gbk = [f for f in listdir('data/antismash-result/') if f.endswith('.gbk')]
gbk.sort()
gbk.pop(0)

# insert gbk path
df_summary.insert(7, "gbk", gbk, True) 


r = []
c = []
for i in df_summary.index:
    region = df_summary.Region[i][-5:].split('.')
    region[0] = re.sub("[^0-9]", "", region[0])
    r.append(region[0])
    c.append(region[1])

# insert gbk path
df_summary.insert(8, "region", r, True) 
df_summary.insert(9, "cluster", c, True) 
df_summary

Region          Type     From       To  \
0     Region&nbsp1.1     NRPS-like   436925   476824   
1     Region&nbsp1.2  NRPS,terpene   508783   549788   
2     Region&nbsp1.3          NRPS   677184   724647   
3     Region&nbsp1.4     NRPS-like  1145545  1189943   
4     Region&nbsp1.5       terpene  1317169  1339303   
..               ...           ...      ...      ...   
74   Region&nbsp73.2     NRPS-like    76174   113391   
75   Region&nbsp85.1         T1PKS    25348    59154   
76   Region&nbsp95.1          NRPS      632    43824   
77   Region&nbsp98.1          NRPS        1    39059   
78  Region&nbsp113.1         T1PKS        1    14679   

   Most similar known cluster Most similar known cluster.1 Similarity  \
0                         NaN                          NaN        NaN   
1                         NaN                          NaN        NaN   
2                         NaN                          NaN        NaN   
3                         NaN                          NaN        NaN   
4                         NaN                          NaN        NaN   
..                        ...                          ...        ...   
74                        NaN                          NaN        NaN   
75                monacolin K                   Polyketide        33%   
76                        NaN                          NaN        NaN   
77              aspercryptins                          NRP        40%   
78                        NaN                          NaN        NaN   

                         gbk region cluster  
0   KZ824267.1.region001.gbk      1       1  
1   KZ824267.1.region002.gbk      1       2  
2   KZ824267.1.region003.gbk      1       3  
3   KZ824267.1.region004.gbk      1       4  
4   KZ824267.1.region005.gbk      1       5  
..                       ...    ...     ...  
74  KZ824339.1.region002.gbk     73       2  
75  KZ824351.1.region001.gbk     85       1  
76  KZ824361.1.region001.gbk     95       1  
77  KZ824364.1.region001.gbk     98       1  
78  KZ824379.1.region001.gbk    113       1  

[79 rows x 10 columns]

In [6]:
# statistics
df_summary.region.describe()

count     79
unique    41
top        1
freq       6
Name: region, dtype: object

In [8]:
df_count = pd.DataFrame(columns=df_summary.Type.unique())
for i in df_summary.region.unique():
    x = df_summary[df_summary['region'] == i].Type.value_counts().to_frame()
    x = x.rename(columns={'Type':i})
    #x.insert(1, "Contigs", np.array(i))
    #print(x)
    df_count = df_count.append(x.transpose())
df_count = df_count.fillna(0)

In [97]:
import seaborn as sns; sns.set()

%matplotlib inline
g = sns.heatmap(df_count.transpose(), annot=False, cbar_kws={"orientation":"vertical"})
g.set_xticklabels(g.get_xticklabels(), rotation = 90)
g.xaxis.set_ticks_position('top')
g.xaxis.set_label_position('top')
g.set_xlabel('Biosynthetic Gene Clusters')

Text(0.5, 12.5, 'Biosynthetic Gene Clusters')

In [96]:
sns.set_palette("Spectral", n_colors=18)
df_bar = df_count.sort_index(axis=1)
ax = df_bar.plot(kind='barh', stacked=True, legend=False, figsize=(6,12))
ax.set_xlabel("Biosynthetic Gene Clusters")
ax.set_ylabel("Regions")
chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.6, chartBox.height])
ax.legend(loc='upper right', bbox_to_anchor=(2.2, 0.8), shadow=True, ncol=1)
ax.set_xlim([0, 6])
#fig = ax.get_figure()
#fig.savefig('region_bgcs.png')

(0.0, 6.0)

In [92]:
ax = df_summary.Type.value_counts().sort_values().plot(kind='barh', legend=False, figsize=(10,10))
ax.set_xlim([0, 18])
#ax.legend(loc='upper right', ncol=1)
#fig = df_summary.Type.value_counts().plot(kind='barh',  figsize=(15, 10), fontsize=18).get_figure()

#fig.savefig('test.png')

(0.0, 18.0)

In [12]:
# Filter for T1PKS
t1pks = df_summary[df_summary['Type'] == 'T1PKS']
t1pks = t1pks[t1pks['Similarity'] != '100%']
t1pks

Region   Type     From       To  \
5     Region&nbsp1.6  T1PKS  1845567  1893829   
8     Region&nbsp2.3  T1PKS   439045   486192   
17    Region&nbsp4.5  T1PKS  1342463  1378538   
21    Region&nbsp6.4  T1PKS   931411   979470   
23    Region&nbsp9.1  T1PKS   327024   375598   
28   Region&nbsp12.1  T1PKS   643222   684146   
29   Region&nbsp13.1  T1PKS   135158   180413   
30   Region&nbsp14.1  T1PKS    13863    61566   
42   Region&nbsp20.3  T1PKS   258970   306744   
50   Region&nbsp28.1  T1PKS   171418   218949   
54   Region&nbsp36.1  T1PKS    13664    61694   
60   Region&nbsp44.2  T1PKS    64475   112544   
68   Region&nbsp68.2  T1PKS    60366   107858   
75   Region&nbsp85.1  T1PKS    25348    59154   
78  Region&nbsp113.1  T1PKS        1    14679   

                           Most similar known cluster  \
5                                                 NaN   
8                                                 NaN   
17                                        neosartorin   
21                                                NaN   
23                                                NaN   
28                                                NaN   
29                                            patulin   
30                                       oxaleimide C   
42                                                NaN   
50                                                NaN   
54  ankaflavin / monascin / rubropunctatine / mona...   
60                                                NaN   
68                                                NaN   
75                                        monacolin K   
78                                                NaN   

   Most similar known cluster.1 Similarity                       gbk region  \
5                           NaN        NaN  KZ824267.1.region006.gbk      1   
8                           NaN        NaN  KZ824268.1.region003.gbk      2   
17                   Polyketide        57%  KZ824270.1.region005.gbk      4   
21                          NaN        NaN  KZ824272.1.region004.gbk      6   
23                          NaN        NaN  KZ824275.1.region001.gbk      9   
28                          NaN        NaN  KZ824278.1.region001.gbk     12   
29  Polyketide:Iterative type I        66%  KZ824279.1.region001.gbk     13   
30             NRP + Polyketide        35%  KZ824280.1.region001.gbk     14   
42                          NaN        NaN  KZ824286.1.region003.gbk     20   
50                          NaN        NaN  KZ824294.1.region001.gbk     28   
54  Polyketide:Iterative type I        12%  KZ824302.1.region001.gbk     36   
60                          NaN        NaN  KZ824310.1.region002.gbk     44   
68                          NaN        NaN  KZ824334.1.region002.gbk     68   
75                   Polyketide        33%  KZ824351.1.region001.gbk     85   
78                          NaN        NaN  KZ824379.1.region001.gbk    113   

   cluster  
5        6  
8        3  
17       5  
21       4  
23       1  
28       1  
29       1  
30       1  
42       3  
50       1  
54       1  
60       2  
68       2  
75       1  
78       1

In [13]:
# extract index
index = df_summary.index.tolist()

# Find Phenylalanine related genes
for i in index:
    print_cluster(i, df_summary, "CDS", 'product', 'phenylalanine')

index :51
['30', '1']
https://dev.fungismash.secondarymetabolites.org/upload/fungi-28f77eb4-83c8-400e-8b6c-9f06a0c99790/index.html#r30c1
phenylalanine ammonia-lyase
index :68
['68', '2']
https://dev.fungismash.secondarymetabolites.org/upload/fungi-28f77eb4-83c8-400e-8b6c-9f06a0c99790/index.html#r68c2
phenylalanine ammonia-lyase


In [14]:
# extract index
index = [51, 68]

# print functions in cluster
for i in index:
    print_cluster(i, df_summary, "CDS", 'gene_functions')

51 30-1 biosynthetic (rule-based-clusters) indole: dmat
51 30-1 biosynthetic-additional (rule-based-clusters) p450
51 30-1 biosynthetic-additional (smcogs) SMCOG1034:cytochrome P450 (Score: 283.1; E-value: 6.9e-86)
68 68-2 biosynthetic-additional (rule-based-clusters) Peptidase_S8
68 68-2 biosynthetic-additional (smcogs) SMCOG1075:alkaline serine protease, subtilase family (Score: 164.8; E-value: 6.8e-50)
68 68-2 biosynthetic-additional (rule-based-clusters) Abhydrolase_6
68 68-2 biosynthetic (rule-based-clusters) T1PKS: itr_KS
68 68-2 biosynthetic (rule-based-clusters) T1PKS: PKS_AT
68 68-2 biosynthetic-additional (rule-based-clusters) PP-binding
68 68-2 biosynthetic-additional (rule-based-clusters) adh_short
68 68-2 biosynthetic-additional (smcogs) SMCOG1021:malonyl CoA-acyl carrier protein transacylase (Score: 121.7; E-value: 6.4e-37)
68 68-2 biosynthetic-additional (rule-based-clusters) AMP-binding
68 68-2 biosynthetic-additional (smcogs) SMCOG1002:AMP-dependent synthetase and liga

In [15]:
# extract index
index = [68]

# print functions in cluster
for i in index:
    print_cluster(i, df_summary, "PFAM_domain", 'description')

68 68-2 Subtilase family
68 68-2 Aromatic amino acid lyase
68 68-2 Thioesterase domain
68 68-2 Methyltransferase domain
68 68-2 Beta-ketoacyl synthase, N-terminal domain
68 68-2 Beta-ketoacyl synthase, C-terminal domain
68 68-2 Ketoacyl-synthetase C-terminal extension
68 68-2 Acyl transferase domain
68 68-2 Polyketide synthase dehydratase
68 68-2 Methyltransferase domain
68 68-2 KR domain
68 68-2 Phosphopantetheine attachment site
68 68-2 AMP-binding enzyme
68 68-2 AMP-binding enzyme C-terminal domain
68 68-2 Transmembrane amino acid transporter protein
68 68-2 Domain of unknown function (DUF3328)
68 68-2 Domain of unknown function (DUF3328)
68 68-2 Domain of unknown function (DUF3328)
68 68-2 ABC-2 type transporter
68 68-2 ABC transporter
68 68-2 CDR ABC transporter
68 68-2 ABC-2 type transporter
68 68-2 ABC transporter
68 68-2 ABC-transporter N-terminal


In [16]:
# compare MIBIG for interesting cluster
df[mibig(68)]

Reference  T1PKS  Similarity score                 Type  \
0  BGC0000046.1    NaN              0.33           Polyketide   
1  BGC0001068.1    NaN              0.30  Terpene, Polyketide   
2  BGC0001858.1    NaN              0.27           Polyketide   
3  BGC0001280.1    NaN              0.27           Polyketide   
4  BGC0001264.1    NaN              0.27           Polyketide   
5  BGC0002067.1    NaN              0.23      NRP, Polyketide   
6  BGC0001141.1    NaN              0.23           Polyketide   
7  BGC0001402.1    NaN              0.22                  NRP   
8  BGC0001254.1    NaN              0.22           Polyketide   
9  BGC0000003.1    NaN              0.22           Polyketide   

                                         Compound(s)  \
0                                          depudecin   
1                                     pyripyropene A   
2  alternapyrone B, alternapyrone C, alternapyron...   
3                           betaenone C, betaenone A   
4              betaenone A, betaenone B, betaenone C   
5                                          atpenin B   
6                           4-epi-15-epi-brefeldin A   
7  cyclochlorotine, hydroxy-cyclochlorotine, cycl...   
8                                       ACT-Toxin II   
9                                           AF-toxin   

                               Organism  
0               Alternaria brassicicola  
1  unidentified unclassified sequences.  
2         Parastagonospora nodorum SN15  
3                           Phoma betae  
4                           Phoma betae  
5            Penicillium oxalicum 114-2  
6              Penicillium brefeldianum  
7                Talaromyces islandicus  
8                  Alternaria alternata  
9                  Alternaria alternata